In [8]:
# The 6.00 Word Game

import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

# -----------------------------------
# Helper code
# (you don't need to understand this helper code)

WORDLIST_FILENAME = "words1.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

# (end of helper code)
# -----------------------------------

#
# Problem #1: Scoring a word
#
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    a = 0
    b = []
    for c in word:
        a += SCRABBLE_LETTER_VALUES[c]
    a = a * len(word)
    if len(word) == n:
        a += 50
    return a



#
# Problem #2: Make sure you understand how this function works and what it does!
#
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()                             # print an empty line

#
# Problem #2: Make sure you understand how this function works and what it does!
#
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3
    
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand

#
# Problem #2: Update a hand by removing letters
#
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    a = hand.copy()
    for b in word:
        a[b] -= 1
    return a



#
# Problem #3: Test word validity
#
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    a = hand.copy()
    if word not in wordList:
        return False
    else:
        
        try:
            for b in word:
                a[b] = a[b] - 1
                if a[b] == 0:
                    del a[b]
        except KeyError:
            return False
        else:
            return True
        


#
# Problem #4: Playing a hand
#

def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    a = 0
    for b in hand.keys():
        a += hand[b]
    return a



def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # Keep track of the total score
    score = 0
    # As long as there are still letters left in the hand:
    while True:
        # Display the hand
        print('Current Hand:  ', end='')
        displayHand(hand)
        # Ask user for input
        a = input('Enter word, or a "." to indicate that you are finished: ')
        # If the input is a single period:
        if a == '.':
            # End the game (break out of the loop)
            print('Goodbye! Total score: ' + str(score) + ' points.')
            break
            
        # Otherwise (the input is not a single period):
        else:
            # If the word is not valid:
            if isValidWord(a, hand, wordList) == False:
                # Reject invalid word (print a message followed by a blank line)
                print('Invalid word, please try again.')
                print('')
            # Otherwise (the word is valid):
            else:
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line
                score += getWordScore(a, n)
                print('"' + a + '" earned ' + str(getWordScore(a, n)) + ' points. Total: ' + str(score) + ' points')
                print('')
                # Update the hand 
                hand = updateHand(hand, a)

            if calculateHandlen(hand) == 0:
                print('Run out of letters. Total score: ' + str(score) + ' points.')
                break
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
        

#
# Problem #5: Playing a game
# 

def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1    
    """
    HAND_SIZE = 4
    b = 0
    while True:
        a = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if a == 'n':
            c = dealHand(HAND_SIZE)
            b += 1
            playHand(c, wordList, HAND_SIZE)
        elif a == 'r':
            if b == 0:
                print('You have not played a hand yet. Please play a new hand first!')
                print('')
            else:
                playHand(c, wordList, HAND_SIZE)
        elif a == 'e':
            break
        else:
            print('Invalid command.')
                

   



#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)


Loading word list from file...
   83667 words loaded.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  e b y q 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  o q f s 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  o c n p 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  o p p h 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  i y c r 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
E

Current Hand:  a w p d 
Enter word, or a "." to indicate that you are finished: pwad
Invalid word, please try again.

Current Hand:  a w p d 
Enter word, or a "." to indicate that you are finished: dwap
Invalid word, please try again.

Current Hand:  a w p d 
Enter word, or a "." to indicate that you are finished: pawd
Invalid word, please try again.

Current Hand:  a w p d 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  i h r x 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  i h x g 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  e z c g 
Enter word, or a "." to indicate that you are finished:

Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  e r g v 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  i j w n 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  a h d n 
Enter word, or a "." to indicate that you are finished: dhan
Invalid word, please try again.

Current Hand:  a h d n 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 0 points.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Current Hand:  u b p h 
Enter word, or a "." to indicate that you are finished: phub
Invalid word, please try again.

Current Hand:  u b p h 
Enter word, or a "." to indicate that you are finished: bhup]
Invalid word, please try again.

Cur

In [64]:
#
# Test code
# You don't need to understand how this test code works (but feel free to look it over!)

# To run these tests, simply run this file (open up in your IDE, then run the file as normal)

def test_getWordScore():
    """
    Unit test for getWordScore
    """
    failure=False
    # dictionary of words and scores
    words = {("", 7):0, ("it", 7):4, ("was", 7):18, ("scored", 7):54, ("waybill", 7):155, ("outgnaw", 7):127, ("fork", 7):44, ("fork", 4):94}
    for (word, n) in words.keys():
        score = getWordScore(word, n)
        if score != words[(word, n)]:
            print("FAILURE: test_getWordScore()")
            print("\tExpected", words[(word, n)], "points but got '" + str(score) + "' for word '" + word + "', n=" + str(n))
            failure=True
    if not failure:
        print("SUCCESS: test_getWordScore()")

# end of test_getWordScore


def test_updateHand():
    """
    Unit test for updateHand
    """
    # test 1
    handOrig = {'a':1, 'q':1, 'l':2, 'm':1, 'u':1, 'i':1}
    handCopy = handOrig.copy()
    word = "quail"

    hand2 = updateHand(handCopy, word)
    expectedHand1 = {'l':1, 'm':1}
    expectedHand2 = {'a':0, 'q':0, 'l':1, 'm':1, 'u':0, 'i':0}
    if hand2 != expectedHand1 and hand2 != expectedHand2:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tReturned: ", hand2, "\n\t-- but expected:", expectedHand1, "or", expectedHand2)

        return # exit function
    if handCopy != handOrig:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of updateHand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function
        
    # test 2
    handOrig = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    handCopy = handOrig.copy()
    word = "evil"

    hand2 = updateHand(handCopy, word)
    expectedHand1 = {'v':1, 'n':1, 'l':1}
    expectedHand2 = {'e':0, 'v':1, 'n':1, 'i':0, 'l':1}
    if hand2 != expectedHand1 and hand2 != expectedHand2:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tReturned: ", hand2, "\n\t-- but expected:", expectedHand1, "or", expectedHand2)

        return # exit function

    if handCopy != handOrig:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of updateHand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function

    # test 3
    handOrig = {'h': 1, 'e': 1, 'l': 2, 'o': 1}
    handCopy = handOrig.copy()
    word = "hello"

    hand2 = updateHand(handCopy, word)
    expectedHand1 = {}
    expectedHand2 = {'h': 0, 'e': 0, 'l': 0, 'o': 0}
    if hand2 != expectedHand1 and hand2 != expectedHand2:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tReturned: ", hand2, "\n\t-- but expected:", expectedHand1, "or", expectedHand2)
        
        return # exit function

    if handCopy != handOrig:
        print("FAILURE: test_updateHand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of updateHand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function

    print("SUCCESS: test_updateHand()")

# end of test_updateHand

def test_isValidWord(wordList):
    """
    Unit test for isValidWord
    """
    failure=False
    # test 1
    word = "hello"
    handOrig = getFrequencyDict(word)
    handCopy = handOrig.copy()

    if not isValidWord(word, handCopy, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected True, but got False for word: '" + word + "' and hand:", handOrig)

        failure = True

    # Test a second time to see if wordList or hand has been modified
    if not isValidWord(word, handCopy, wordList):
        print("FAILURE: test_isValidWord()")

        if handCopy != handOrig:
            print("\tTesting word", word, "for a second time - be sure you're not modifying hand.")
            print("\tAt this point, hand ought to be", handOrig, "but it is", handCopy)

        else:
            print("\tTesting word", word, "for a second time - have you modified wordList?")
            wordInWL = word in wordList
            print("The word", word, "should be in wordList - is it?", wordInWL)

        print("\tExpected True, but got False for word: '" + word + "' and hand:", handCopy)

        failure = True


    # test 2
    hand = {'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u':1}
    word = "rapture"

    if  isValidWord(word, hand, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)

        failure = True        

    # test 3
    hand = {'n': 1, 'h': 1, 'o': 1, 'y': 1, 'd':1, 'w':1, 'e': 2}
    word = "honey"

    if  not isValidWord(word, hand, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected True, but got False for word: '"+ word +"' and hand:", hand)

        failure = True                        

    # test 4
    hand = {'r': 1, 'a': 3, 'p': 2, 't': 1, 'u':2}
    word = "honey"

    if  isValidWord(word, hand, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        
        failure = True

    # test 5
    hand = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    word = "evil"
    
    if  not isValidWord(word, hand, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected True, but got False for word: '" + word + "' and hand:", hand)
        
        failure = True
        
    # test 6
    word = "even"

    if  isValidWord(word, hand, wordList):
        print("FAILURE: test_isValidWord()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        print("\t(If this is the only failure, make sure isValidWord() isn't mutating its inputs)")
        
        failure = True        

    if not failure:
        print("SUCCESS: test_isValidWord()")


wordList = loadWords()
print("----------------------------------------------------------------------")
print("Testing getWordScore...")
test_getWordScore()
print("----------------------------------------------------------------------")
print("Testing updateHand...")
test_updateHand()
print("----------------------------------------------------------------------")
print("Testing isValidWord...")
test_isValidWord(wordList)
print("----------------------------------------------------------------------")
print("All done!")


Loading word list from file...
   83667 words loaded.
----------------------------------------------------------------------
Testing getWordScore...
SUCCESS: test_getWordScore()
----------------------------------------------------------------------
Testing updateHand...
SUCCESS: test_updateHand()
----------------------------------------------------------------------
Testing isValidWord...
SUCCESS: test_isValidWord()
----------------------------------------------------------------------
All done!


In [75]:
wordList = loadWords()
playHand({'h':1, 'i':1, 'c':1, 'z':1, 'm':2, 'a':1}, wordList, 7)

Loading word list from file...
   83667 words loaded.
Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: zam
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: maz
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: haz
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: zam
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: zac
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: caz
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished: chaz
Invalid word, please try again.

Current Hand:  h i c z m m a 
Enter word, or a "." to indicate that you are finished